<a href="https://colab.research.google.com/github/Abeszz/SC4002-NLP-Assignment/blob/main/SC4002.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installation & Requirements :

In [1]:
pip install datasets


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install nltk


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install numpy


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install npm


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install torch


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [6]:
!wget https://nlp.stanford.edu/data/glove.6B.zip
!unzip glove.6B.zip

zsh:1: command not found: wget
unzip:  cannot find or open glove.6B.zip, glove.6B.zip.zip or glove.6B.zip.ZIP.


In [7]:
import os
import nltk
import sys

env_base_path = sys.prefix
nltk_path = os.path.join(env_base_path, 'nltk_data')
nltk.download('punkt', nltk_path)
nltk.download('punkt_tab', nltk_path)

[nltk_data] Downloading package punkt to /opt/anaconda3/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /opt/anaconda3/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

# **PART 0. Dataset Preparation**

In [8]:
from datasets import load_dataset
dataset = load_dataset('rotten_tomatoes')

In [9]:
train_dataset = dataset['train']
validation_dataset = dataset['validation']
test_dataset = dataset['test']

# **Part 1. Preparing Word Embedding**

In [10]:
vocabulary = None
vocab_size = None
embedding_matrix = None
oov_words = [] # Array to track OOV words
UNKNOWN_TOKEN = '<UNKNOWN>'  # Special token for OOV words

glove_model_file = 'glove.6B.300d.txt'
glove_model_dimension = 300

In [11]:
from nltk.tokenize import word_tokenize
from collections import Counter

def tokenize_sentence(sentence):
    # Tokenize each sentence using NLTK's word_tokenize
    return word_tokenize(sentence.lower()) # Convert to lowercase for consistency

def tokenize_dataset(dataset):
    global vocabulary, vocab_size
    # Build a vocabulary from the dataset
    vocab_counter = Counter()

    for word in dataset:
        tokens = tokenize_sentence(word['text'])
        vocab_counter.update(tokens)

    # Update the vocabulary list
    vocabulary = list(vocab_counter.keys())
    vocab_size = len(vocabulary)

In [12]:
def load_glove_embeddings(glove_file):
    embeddings_index = {}
    with open(glove_file, 'r', encoding='utf-8') as f:
        for line in f:
            values = line.split()
            word = values[0]
            vector = np.asarray(values[1:], dtype='float32')
            embeddings_index[word] = vector
    return embeddings_index

In [13]:
import numpy as np

def create_embedding_matrix(glove_file_path, glove_model_dimension):
    global vocabulary, embedding_matrix, oov_words
    # Load the GloVe embeddings into a dictionary
    glove_embeddings = load_glove_embeddings(glove_file_path)

    # Create the embedding matrix
    embedding_matrix = np.zeros((vocab_size, glove_model_dimension))  # Initialize the matrix with zeros

    # Initialize the UNKNOWN token with a random vector
    unknown_vector = np.random.normal(scale=0.6, size=(glove_model_dimension,))

    # Fill the embedding matrix
    for idx, word in enumerate(vocabulary):
        if word in glove_embeddings:
            embedding_matrix[idx] = glove_embeddings[word]  # Use the pretrained GloVe vector
        elif word == UNKNOWN_TOKEN:
            embedding_matrix[idx] = unknown_vector  # Do not add UNKNOWN token to oov
        else:
            oov_words.append(word)  # Track OOV words
            embedding_matrix[idx] = unknown_vector

In [14]:
def add_unkown_token():
    global vocabulary, vocab_size, UNKNOWN_TOKEN

    # Check if 'UNK' is already in the vocabulary, if not, add it
    if UNKNOWN_TOKEN not in vocabulary:
        vocabulary.append(UNKNOWN_TOKEN)
        vocab_size += 1


In [15]:
# Tokenize the dataset
tokenize_dataset(train_dataset)

KeyboardInterrupt: 

In [ ]:
# Print the size of Vocabulary
print(f"1(a) Number of Vocabulary words in training dataset = {vocab_size}")

In [ ]:
# 1(c) : Add <UNKNOWN> token and replace all OOV words with it
add_unkown_token()

In [ ]:
# Initialize the glove model with our configuration
create_embedding_matrix(glove_model_file, glove_model_dimension)

In [ ]:
# Print the number of OOV words
print(f"1(b) Number of Out Of Vocabulary words: {len(oov_words)}")

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torch.nn.utils.rnn import pad_sequence
import numpy as np
from nltk.tokenize import word_tokenize
import json

In [ ]:
# Custom collate_fn to handle padding
def collate_fn(batch):
    inputs, labels = zip(*batch)
    inputs = [torch.tensor(x) for x in inputs]
    padded_inputs = pad_sequence(inputs, batch_first=True, padding_value=0)  # Pad sequences
    labels = torch.stack(labels)
    return padded_inputs, labels

In [ ]:
class SentimentRNN(nn.Module):
    def __init__(self, embedding_matrix, hidden_size, output_size, num_layers=1):
        super(SentimentRNN, self).__init__()

        vocab_size, embedding_dim = embedding_matrix.shape

        # Embedding layer using pre-trained embeddings
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.embedding.weight = nn.Parameter(torch.tensor(embedding_matrix, dtype=torch.float32))
        self.embedding.weight.requires_grad = False  # freeze embeddings

        # Simple RNN layer, 1 layer, uni-directional
        self.rnn = nn.RNN(embedding_dim, hidden_size, num_layers=num_layers, batch_first=True)  # Uni-directional RNN with 1 layer

        # RNN layer
        # self.rnn = nn.RNN(embedding_dim, hidden_size, num_layers=2, batch_first=True, bidirectional=True)
        # self.rnn = nn.LSTM(embedding_dim, hidden_size, num_layers=2, batch_first=True, bidirectional=True)
        # self.rnn = nn.GRU(embedding_dim, hidden_size, num_layers=num_layers, batch_first=True, bidirectional=True)

        # Fully connected layer for classification
        self.fc = nn.Linear(hidden_size, output_size)  # Double hidden size for bidirectional RNN

        # Regularization layers
        # self.dropout = nn.Dropout(0.3)  # dropout rate
        # self.batch_norm = nn.BatchNorm1d(hidden_size * 2)  # Add batch normalization
        # self.layer_norm = nn.LayerNorm(hidden_size * 2)  # Add layer normalization

    def forward(self, x):
        embedded = self.embedding(x)
        output, hidden = self.rnn(embedded) # for rnn and gru only two values are returned
        final_output = hidden[-1] # Use the last hidden state for uni-directional RNN
        # output, (hidden, _) = self.rnn(embedded) # for lstm
        # final_output = torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim=1)  # Concatenate forward and backward hidden states
        # final_output = self.batch_norm(final_output)  # Apply batch normalization
        # final_output = self.layer_norm(final_output)  # Apply layer normalization
        # final_output = self.dropout(final_output)     # Apply dropout
        return self.fc(final_output)

In [ ]:
# Dataset class to handle data batching
class TextDataset(Dataset):
    def __init__(self, dataset, vocab):
        self.dataset = dataset
        self.vocab = vocab

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        sentence = self.dataset[idx]['text']
        label = self.dataset[idx]['label']
        tokens = word_tokenize(sentence.lower())
        indices = [self.vocab.index(token) if token in self.vocab else self.vocab.index("<UNKNOWN>") for token in tokens]
        return torch.tensor(indices), torch.tensor(label)

In [ ]:
def get_optimizer(params, model):
    optimizer_type = params["optimizer_type"]
    lr = params["learning_rate"]
    weight_decay = params.get("weight_decay", 0)  # Default to 0 if not specified

    if optimizer_type == "SGD":
        momentum = params.get("momentum", 0)  # Default to 0 if not specified
        return optim.SGD(model.parameters(), lr=lr, momentum=momentum, weight_decay=weight_decay)

    elif optimizer_type == "Adam":
        return optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)

    else:
        raise ValueError(f"Unknown optimizer type: {optimizer_type}")

In [ ]:
def train_model(train_dataset, validation_dataset, vocabulary, embedding_matrix, params_file="hyperparams.txt", optimizer_file="optimizer_params.txt"):
    # Read hyperparameters from file
    with open(params_file, "r") as f:
        params = json.load(f)

    # Load optimizer parameters
    with open(optimizer_file, "r") as f:
        optimizer_params = json.load(f)

    output_size = params['output_size']
    hidden_size = params['hidden_size']
    batch_size = params['batch_size']
    epochs = params['epochs']
    lr = params['learning_rate']
    patience = params['patience']

    model = SentimentRNN(embedding_matrix, hidden_size, output_size)

    # Get the optimizer dynamically based on the configuration in optimizer_file
    optimizer = get_optimizer(optimizer_params, model)

    criterion = nn.CrossEntropyLoss()
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)

    # Create DataLoaders for training and validation with custom collate_fn for padding
    train_loader = DataLoader(TextDataset(train_dataset, vocabulary), batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
    valid_loader = DataLoader(TextDataset(validation_dataset, vocabulary), batch_size=batch_size, collate_fn=collate_fn)

    best_val_accuracy = 0
    epochs_no_improve = 0

    for epoch in range(epochs):
        print(f"Starting Epoch {epoch + 1}/{epochs}")
        model.train()
        running_loss = 0.0
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        print(f'Epoch {epoch+1}, Loss: {running_loss/len(train_loader)}')

        # Validation
        model.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for inputs, labels in valid_loader:
                outputs = model(inputs)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        val_accuracy = 100 * correct / total
        print(f'Validation Accuracy: {val_accuracy}%')

        # Early stopping logic: check if validation accuracy improved
        if val_accuracy > best_val_accuracy:
            best_val_accuracy = val_accuracy
            epochs_no_improve = 0  # Reset the patience counter
            torch.save(model.state_dict(), 'best_model.pt')  # Save the best model's weights
        else:
            epochs_no_improve += 1

        scheduler.step()  # Step the learning rate scheduler

        if epochs_no_improve >= patience:
            print(f'Early stopping at epoch {epoch+1}')
            break

        print("Evaluating model on test set...")
        test_accuracy = evaluate_model_on_test(model, test_dataset, vocabulary)

    return model


In [ ]:
def evaluate_model_on_test(model, test_dataset, vocabulary):
    # Create a DataLoader for the test dataset
    test_loader = DataLoader(TextDataset(test_dataset, vocabulary), batch_size=32, collate_fn=collate_fn)

    model.eval()  # Set the model to evaluation mode
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    # Calculate and print the test accuracy
    test_accuracy = 100 * correct / total
    print(f'Test Accuracy: {test_accuracy}%')
    return test_accuracy